In [67]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@synonly.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

StatementMeta(, , -1, Finished, , Finished)

None: Livy session has failed. Session state: Killed. Session has been cancelled. Source: User.

In [ ]:
files = mssparkutils.fs.ls(bronze_adls)
for file in files:
    print(file.name, file.isDir, file.isFile, file.path, file.size, file.modifyTime)

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
import requests
import json
from datetime import date, timedelta

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)
end_date = date.today()

In [ ]:
start_date, end_date

In [ ]:
# Construct the API URL with start and end dates provided by Data Factory
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Convert data to JSON string
        json_data = json.dumps(data, indent=4)

        # Write the JSON data to ADLS
        # Create an RDD with the JSON string and parallelize it
        rdd = spark.sparkContext.parallelize([json_data])

        # Convert RDD to DataFrame and write to ADLS
        df = spark.read.json(rdd)
        df.limit(3) # To speed up processing for tutorial
        df.write.mode("overwrite").json(file_path)

        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

In [ ]:
data[0]

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
import json

# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
bronze_output = json.dumps(output_data)

# Use mssparkutils.notebook.exit() to pass the JSON output to the pipeline
mssparkutils.notebook.exit(bronze_output)

StatementMeta(, , , Cancelled, , Cancelled)